In [289]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [290]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [296]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 100
n_iter = 13000*2 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [ ]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.NN as nn

class RNN(nn.NN):

    def __init__(self, D, H, char2idx, idx2char):
        self.D = D
        self.H = H
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        super().__init__(D, D, H, None, None, loss='cross_ent', nonlin='relu')

    def initial_state(self):
        return np.zeros((1, self.H))

    def forward(self, X, h, m): # m = self.model
        Wxh, Whh, Why = m['Wxh'], m['Whh'], m['Why']
        bh, by = m['bh'], m['by']

        hprev = h.copy()

        h, h_cache = l.tanh_forward(X @ Wxh + hprev @ Whh + bh)
        y, y_cache = l.fc_forward(h, Why, by)

        cache = X, Whh, h, hprev, y, h_cache, y_cache, Wxh

        return y, h, cache

    def backward(self, dy, dh, cache):
        X, Whh, h, hprev, y, h_cache, y_cache, Wxh = cache

        dh_next = dh.copy()
        
        # Hidden to output gradient
        dh, dWhy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dby = dby.reshape((1, -1))

        # tanh
        dh = l.tanh_backward(dh, h_cache)

        # Hidden gradient
        dbh = dh
        dWhh = hprev.T @ dh
        dWxh = X.T @ dh
        
        dX = dh @ Wxh.T
        dh = dh @ Whh.T

        grad = dict(Wxh=dWxh, Whh=dWhh, Why=dWhy, bh=dbh, by=dby)
        
        return dX, dh, grad

    def _init_model(self, D, C, H):
        self.model = dict(
            Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
            Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
            Why=np.random.randn(H, D) / np.sqrt(C / 2.),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
            )

        self.model2 = dict(
            Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
            Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
            Why=np.random.randn(H, D) / np.sqrt(C / 2.),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
            )

        self.model3 = dict(
            Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
            Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
            Why=np.random.randn(H, D) / np.sqrt(C / 2.),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
            )

        self.model4 = dict(
            Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
            Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
            Why=np.random.randn(H, D) / np.sqrt(C / 2.),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
            )

    def train_step_fwd(self, X_train, h):
        ys, caches, caches2, caches3, caches4 = [], [], [], [], []
        h2 = h.copy()
        h3 = h.copy()
        h4 = h.copy()
        
        for X in X_train:
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            x = X_one_hot.reshape(1, -1)
            y, h, cache = self.forward(x, h, self.model)
            y, h2, cache2 = self.forward(y, h2, self.model2)
            y, h3, cache3 = self.forward(y, h3, self.model3)
            y, h4, cache4 = self.forward(y, h4, self.model4)
            ys.append(y)
            caches.append(cache)
            caches2.append(cache2)
            caches3.append(cache3)
            caches4.append(cache4)
           
        return ys, caches, caches2, caches3, caches4

    def train_step_bwd(self, y_train, ys, caches, caches2, caches3, caches4):
        loss, dys = 0.0, []

        for y_pred, y in zip(ys, y_train):
            loss += loss_fun.cross_entropy(self.model, y_pred, y, lam=0)/ y_train.shape[0]
            dy = loss_fun.dcross_entropy(y_pred, y)
            dys.append(dy)

        # Grads
        dh = np.zeros((1, self.H))
        dh2 = np.zeros((1, self.H))
        dh3 = np.zeros((1, self.H))
        dh4 = np.zeros((1, self.H))
        grads = {key: np.zeros_like(val) for key, val in self.model.items()}
        grads2 = {key: np.zeros_like(val) for key, val in self.model.items()}
        grads3 = {key: np.zeros_like(val) for key, val in self.model.items()}
        grads4 = {key: np.zeros_like(val) for key, val in self.model.items()}
        for t in reversed(range(len(dys))):
            dX, dh4, grad4 = self.backward(dys[t], dh4, caches4[t]) # 4th layer 
            dX, dh3, grad3 = self.backward(dX, dh3, caches3[t]) # 3rd layer 
            dX, dh2, grad2 = self.backward(dX, dh2, caches2[t]) # 2nd layer 
            dX, dh, grad = self.backward(dX, dh, caches[t]) # first layer
            for k in grad.keys():
                grads[k] += grad[k]
                grads2[k] += grad2[k]
                grads3[k] += grad3[k]
                grads4[k] += grad4[k]
        
        return loss, grads, grads2, grad3, grad4

In [ ]:
net = RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [ ]:
import numpy as np
import impl.utils as util
import impl.constant as c
import copy
from sklearn.utils import shuffle as skshuffle

def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha=0.001, mb_size=256, n_iter=2000, print_after=100):
    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    idx = 0
    state = nn.initial_state()
    smooth_loss = -np.log(1.0 / len(set(X_train)))

    M = {k: np.zeros_like(v) for k, v in nn.model.items()}
    R = {k: np.zeros_like(v) for k, v in nn.model.items()}
    M2 = {k: np.zeros_like(v) for k, v in nn.model.items()}
    R2 = {k: np.zeros_like(v) for k, v in nn.model.items()}
    M3 = {k: np.zeros_like(v) for k, v in nn.model.items()}
    R3 = {k: np.zeros_like(v) for k, v in nn.model.items()}    
    M4 = {k: np.zeros_like(v) for k, v in nn.model.items()}
    R4 = {k: np.zeros_like(v) for k, v in nn.model.items()}
    beta1 = .9
    beta2 = .999

    for iter in range(1, n_iter + 1):
        t = iter

        if idx >= len(minibatches):
            idx = 0
            state = nn.initial_state()

        X_mini, y_mini = minibatches[idx]
        idx += 1

        if iter % print_after == 0:
            print('Iter-{} loss: {:.4f}'.format(iter, smooth_loss))

        #             # Testing can be completed once we make sure the training is done and is validated.
        #             sample = nn.test_step_fwd(X_mini[0], state)
        #             print(sample)

        ys, caches, caches2, caches3, caches4 = nn.train_step_fwd(X_mini, state)
        loss, grads, grads2, grads3, grads4 = nn.train_step_bwd(y_mini, ys, caches, caches2, caches3, caches4)
        smooth_loss = 0.999 * smooth_loss + 0.001 * loss

        for k in grads.keys(): #key, value: items
            M[k] = util.exp_running_avg(M[k], grads[k], beta1)
            R[k] = util.exp_running_avg(R[k], grads[k]**2, beta2)

            m_k_hat = M[k] / (1. - beta1**(t))
            r_k_hat = R[k] / (1. - beta2**(t))

            nn.model[k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)
    
        for k in grads.keys(): #key, value: items
            M2[k] = util.exp_running_avg(M2[k], grads2[k], beta1)
            R2[k] = util.exp_running_avg(R2[k], grads2[k]**2, beta2)

            m_k_hat = M2[k] / (1. - beta1**(t))
            r_k_hat = R2[k] / (1. - beta2**(t))

            nn.model2[k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

        for k in grads.keys(): #key, value: items
            M3[k] = util.exp_running_avg(M3[k], grads3[k], beta1)
            R3[k] = util.exp_running_avg(R3[k], grads3[k]**2, beta2)

            m_k_hat = M3[k] / (1. - beta1**(t))
            r_k_hat = R3[k] / (1. - beta2**(t))

            nn.model3[k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

        for k in grads.keys(): #key, value: items
            M4[k] = util.exp_running_avg(M4[k], grads4[k], beta1)
            R4[k] = util.exp_running_avg(R4[k], grads4[k]**2, beta2)

            m_k_hat = M4[k] / (1. - beta1**(t))
            r_k_hat = R4[k] / (1. - beta2**(t))

            nn.model4[k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

    return nn

In [ ]:
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, 
                print_after=print_after)

Iter-1000 loss: 3.6484
Iter-2000 loss: 3.1162
Iter-3000 loss: 2.8041
Iter-4000 loss: 2.5289
Iter-5000 loss: 2.4432
Iter-6000 loss: 2.2466
